#### 1. Load the dependencies


In [1]:
%pip install -r requirements.txt

  Using cached openapi_python_generator-0.5.0-py3-none-any.whl.metadata (5.4 kB)
ERROR: Could not find a version that satisfies the requirement inspect (from versions: none)
ERROR: No matching distribution found for inspect

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


#### 2. Load environment variables


In [2]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)


token = os.environ['HUGGINGFACEHUB_API_TOKEN']

#### 3. Get the openapi.yml from open-mateo


In [3]:
!wget https://raw.githubusercontent.com/open-meteo/open-meteo/main/openapi.yml

--2024-07-02 22:43:10--  https://raw.githubusercontent.com/open-meteo/open-meteo/main/openapi.yml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8002::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13300 (13K) [text/plain]
Saving to: ‘openapi.yml.1’

openapi.yml.1       100%[===================>]  12.99K  --.-KB/s    in 0s      

2024-07-02 22:43:10 (112 MB/s) - ‘openapi.yml.1’ saved [13300/13300]



In [4]:
import yaml
import json

# Read the content of the file
with open('openapi.yml', 'r') as file:
    file_content = file.read()
file_content = file_content.replace("int\n", "number\n")
file_content = file_content.replace("float\n", "number\n")
data = yaml.safe_load(file_content)

data["servers"] = [{"url": "https://api.open-meteo.com"}]

with open('openapi.json', 'w') as file:
    json_content = json.dump(data, file)

#### 4. Import the neccessary function `get_v1forecast` from open-mateo


In [5]:
!openapi-python-generator openapi.json ./api_specification_main/

zsh:1: command not found: openapi-python-generator


In [6]:
from api_specification_main.services.WeatherForecastAPIs_service\
    import get_v1forecast

#### 5. Prompt


In [7]:
user_query = "Hey how is the current weather and windspeed in Charlotte?"

In [8]:
import inspect
signature = inspect.signature(get_v1forecast)
print(signature)
docstring = \
    '''
Requires the latitude and longitude.
Set current_weather to True to get the weather.
Set hourly or daily based on preference.
Return ONLY the function call with the latitude and longitude of the location and no other explanation so I can eval the function directly.
Example "get_v1forecast(latitude=40.7128, longitude=-74.0060, current_weather=True)" as response ONLY, no other text.
Respond in JSON mode with function under the key "result" and value as function string.

'''

prompt = \
    f'''
Function:
{get_v1forecast.__name__}{signature}
"""{docstring}"""

User Query: {user_query}<human_end>'''

print(prompt)

(latitude: float, longitude: float, hourly: Optional[List[str]] = None, daily: Optional[List[str]] = None, current_weather: Optional[bool] = None, temperature_unit: Optional[str] = None, wind_speed_unit: Optional[str] = None, timeformat: Optional[str] = None, timezone: Optional[str] = None, past_days: Optional[int] = None, api_config_override: Optional[api_specification_main.api_config.APIConfig] = None) -> Dict[str, Any]

Function:
get_v1forecast(latitude: float, longitude: float, hourly: Optional[List[str]] = None, daily: Optional[List[str]] = None, current_weather: Optional[bool] = None, temperature_unit: Optional[str] = None, wind_speed_unit: Optional[str] = None, timeformat: Optional[str] = None, timezone: Optional[str] = None, past_days: Optional[int] = None, api_config_override: Optional[api_specification_main.api_config.APIConfig] = None) -> Dict[str, Any]
"""
Requires the latitude and longitude.
Set current_weather to True to get the weather.
Set hourly or daily based on prefe

#### 6. Use Ollama with lllama3 model and Function Calling


In [9]:
from langchain.llms import Ollama
llm = Ollama(temperature=0, model="llama3")

### Function Calling

The LLM does not have the latest info. However, it knows the function calling based on the api spec provided to it with the prompt. So it returns the method signature that we can use to call the function


In [10]:
response = llm.invoke(prompt)

In [11]:
print(response)

{
"result": "get_v1forecast(latitude=35.2278, longitude=-80.8431, current_weather=True)"
}


#### 7. Print the Output


In [12]:
r = json.loads(response)
r

{'result': 'get_v1forecast(latitude=35.2278, longitude=-80.8431, current_weather=True)'}

In [13]:
data = eval(r["result"])

In [14]:
temperature = data['current_weather']['temperature']
windspeed = data['current_weather']['windspeed']
winddirection = data['current_weather']['winddirection']
is_day = data['current_weather']['is_day']
time = data['current_weather']['time']

# Determine day or night
day_or_night = 'Day' if is_day else 'Night'

# Print the weather information nicely
print(f"Weather Information:")
print(f"Time: {time}")
print(f"Temperature: {temperature}°C")
print(f"Wind Speed: {windspeed} km/h")
print(f"Wind Direction: {winddirection}°")
print(f"Day/Night: {day_or_night}")

Weather Information:
Time: 2024-07-03T02:30
Temperature: 24.1°C
Wind Speed: 6.9 km/h
Wind Direction: 81°
Day/Night: Night
